In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export text_norm

Cells will be exported to text2speech.text_norm,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Text Normalization
> Functions used for TTS Dataset Preparation

In [ ]:
%nbdev_export
import re

In [ ]:
#hide
from fastcore.test import *
from nbdev.showdoc import *

## Functions for Pipeline

In [ ]:
%nbdev_export
def collapse_whitespace(text: str) -> str:
    "Replace multiple various whitespaces with a single space, strip leading and trailing spaces."
    
    return re.sub(r'[\s\ufeff\u200b\u2060]+', ' ', text).strip()

In [ ]:
test_eq(collapse_whitespace(
    chr(int("0xfeff", 16)) + # zero width no-break space
    chr(int("0x200b", 16)) + # zero width space
    chr(int("0x202f", 16)) + # narrow no-break space
    chr(int("0x2060", 16)) + # word joiner
    chr(int("0x3000", 16)) + # ideographic space
    chr(int("0xa0"  , 16)) + # no-break space
    "\t\n 1 2   3     4     5       \t\r\n"),
    "1 2 3 4 5")

In [ ]:
%nbdev_export
def lowercase(text: str) -> str:
    "Convert `text` to lower case."
    
    return text.lower()

In [ ]:
test_eq(lowercase('ПрИвеТ, ЧуВАК!'), 
                  'привет, чувак!')

In [ ]:
%nbdev_export
def check_no_numbers(text: str) -> list:
    "Return a list of digits, or empty list, if not found."
    
    return re.findall(r'(\d+)', text)

In [ ]:
test_eq(check_no_numbers('Цифры есть 1 12 13.4'), ['1', '12', '13', '4'])
test_eq(check_no_numbers('Цифр нет'), [])

In [ ]:
%nbdev_export_internal
_specials = [(re.compile(f'{x[0]}'), x[1]) for x in [
    (r'\(?\d\d[:.]\d\d\)?', ''),  # timestamps
    (r'!\.{1,}', '!'), # !. -> !
    (r'\?\.{1,}', '?'),# ?. -> ?
    (r'\/', ''),
    (r'[\*\_]', '')
    ]]

In [ ]:
%nbdev_export
def remove_specials(text: str, purge_digits: bool=None) -> str:
    "Replace predefined in `_specials` sequence of characters"
    
    for regex, replacement in _specials:
        text = re.sub(regex, replacement, text)
    if purge_digits:
        text = re.sub(r'\d', '', text)
    return text

In [ ]:
%nbdev_export
def purge_dots(text, purgedots=False):
    "If `purgedots`, `...`|`…` will be purged. Else replaced with `.`"
    text = re.sub(r'\s(…)', ' ', text)
    replacement = '' if purgedots else '.'
    text = re.sub(r'…', replacement, text)
    text = re.sub(r'\.{3}', replacement, text)
    text = re.sub(r'\.{2}', '', text)   # pause .. removed
    return text

In [ ]:
test_eq(purge_dots("Word..."), 'Word.')
test_eq(purge_dots("Word…",), 'Word.')

test_eq(purge_dots("Word...", purgedots=True), 'Word')
test_eq(purge_dots("Word…",   purgedots=True), 'Word')

test_eq(purge_dots(" …Word",), ' Word')

test_eq(purge_dots("Word..",), 'Word')

test_eq(purge_dots('Многоточие... Многоточие… … …Многоточие'),
                        'Многоточие. Многоточие.  Многоточие')

In [ ]:
test_eq(remove_specials('Метки времени 01:12 или 01.01, (01:12) или (01.01) удаляем.'),
                        'Метки времени  или ,  или  удаляем.')
test_eq(remove_specials('Ой!. Ага?. / Стоп.'), 'Ой! Ага?  Стоп.')
test_eq(remove_specials('*США* _Френсис_'), 'США Френсис')

In [ ]:
%nbdev_export
_abbreviations = [(re.compile(f'\\b{x[0]}', re.IGNORECASE), x[1]) for x in [
  (r'т\.е\.', 'то есть'),
  (r'т\.к\.', 'так как'),
  (r'и т\.д\.', 'и так далее.'),
  (r'и т\.п\.', 'и тому подобное.')
]]

In [ ]:
%nbdev_export
def expand_abbreviations(text: str) -> str:
    "`expand_abbreviations()` defined in `_abbreviations`"
    
    for regex, replacement in _abbreviations:
        text = re.sub(regex, replacement, text)
    return text

In [ ]:
test_eq(
    expand_abbreviations('Привет Джон, т.е. Иван. Т.к. русский. И т.д. И т.п.'),
                         'Привет Джон, то есть Иван. так как русский. и так далее. и тому подобное.') 

In [ ]:
%nbdev_export
def unify_dash_hyphen(text: str) -> str:
    "Unify dash and hyphen symbols -- replace with emdash or hyphen, separate with space."
    
    text = re.sub('[\u2212\u2012\u2014]', '\u2013', text) # replace minus sign, figure dash, em dash with en dash
    text = re.sub('[\u2010\u2011]', '\u002d', text)  # hyphen, non-breaking hyphen
    text = re.sub('\s*?(\u2013)\s*?',' \g<1> ',text)
    return text

In [ ]:
test_eq(unify_dash_hyphen(
    chr(int("2212",16))+ # minus sign
    chr(int("2012",16))+ # figure dash
    chr(int("2010",16))+ # hyphen
    chr(int("2011",16))),# non-breaking hyphen
    (" "+chr(int("2013",16))+" ")*2+chr(int("2d",16))*2)

In [ ]:
test_eq(unify_dash_hyphen('Я '+chr(int("2013",16))+ 'Джейми Кейлер'),'Я – Джейми Кейлер')
test_eq(unify_dash_hyphen('Я' +chr(int("2013",16))+ 'Джейми Кейлер'),'Я – Джейми Кейлер')
test_eq(collapse_whitespace(unify_dash_hyphen('Я' +chr(int("2013",16))+' Джейми Кейлер')),'Я – Джейми Кейлер')

In [ ]:
%nbdev_export
def rm_quot_marks(text: str) -> str:
    """Remove quotation marks from `text`."""
    # \u0022\u0027\u00ab\u00bb\u2018\u2019\u201a\u201b\u201c\u201d\u201e\u201f\u2039\u203a\u276e\u276f\u275b\u275c\u275d\u275e\u275f\u2760\u2e42\u301d\u301e\u301f
    return re.sub(r'["\'«»‘’‚‛“”„‟‹›❮❯❛❜❝❞❟❠]','',text)   

In [ ]:
test_eq(rm_quot_marks('"\'«»‘’‚‛“”„‟‹›❮❯❛❜❝❞❟❠'),'')

## Pipelines

In [ ]:
%nbdev_export_and_show
def basic_cleaner(text: str) -> str:
    "Basic pipeline: lowercase and collapse whitespaces."
    text = lowercase(text)
    text = collapse_whitespace(text)
    return text

In [ ]:
test_eq(basic_cleaner(
    'Привет   Джон, т.е. Иван, т.к. русский. И т.д.   и т.п.'),
    'привет джон, т.е. иван, т.к. русский. и т.д. и т.п.')

In [ ]:
%nbdev_export_and_show
def russian_cleaner(text, purge_digits=True, _purge_dots=False):
    "Pipeline for cleaning Russian text."
    text = lowercase(text)
    text = expand_abbreviations(text)
    text = remove_specials(text, purge_digits=purge_digits)
    text = purge_dots(text,purgedots=_purge_dots)
    text = unify_dash_hyphen(text)
    text = rm_quot_marks(text)
    text = collapse_whitespace(text)
    return text

In [ ]:
test_eq(russian_cleaner(
        'Привет «Джон», т.е.     Иван, т.к. русский... И т.д. и т.п. Ой!. Ага?. / "Стоп"..'),
        'привет джон, то есть иван, так как русский. и так далее. и тому подобное. ой! ага? стоп')

## Sentences Tokenizer

In [ ]:
import nltk

In [ ]:
fname = '/home/condor/git/cyrillica/i-ish.txt'
with open(fname) as f:
    text = f.read()

In [ ]:
text

'\ufeff\n\n\n\n\n\n\n\nНам кажется, что мы знаем историю Америки, но в рассказах о великих шоуменах больше мифов, чем истины. Взять мастера исчезновений, Гудини. Думаете, он снимал кандалы, выворачивая руки? Плохо думаете. А страшный номер с водой не имеет отношения к его смерти. И что там про тяжеловеса шоубизнеса, ПиТи Барнума? Он не говорил «каждую минуту рождается простофиля». Вы не поверите, узнав, чьи это слова. Я – Джейми Кейлер. Сегодня мы отринем легенды и докопаемся до истины!\n\n\nАМЕРИКА:  ФАКТЫ  И  ДОМЫСЛЫ\n\n\nБлагодаря удивительному таланту снимать с себя любые кандалы Гарри Гудини стал одним из самых известных артистов двадцатого века. Он выбирался из сейфов, банковских камер, фортепиано, чугунных котлов, гигантского футбольного мяча, даже из скелета таинственного морского чудища. Осталась лишь одна сеть, откуда он не выбрался: паутина легенд, опутавшая его имя.\n\n\nФокусы скучнеют, когда вы знаете их объяснение. Оказывается, всё просто. Неинтересно. \n\n\nРоб Уильямс,

In [ ]:
russian_cleaner(text)

'нам кажется, что мы знаем историю америки, но в рассказах о великих шоуменах больше мифов, чем истины. взять мастера исчезновений, гудини. думаете, он снимал кандалы, выворачивая руки? плохо думаете. а страшный номер с водой не имеет отношения к его смерти. и что там про тяжеловеса шоубизнеса, пити барнума? он не говорил каждую минуту рождается простофиля. вы не поверите, узнав, чьи это слова. я – джейми кейлер. сегодня мы отринем легенды и докопаемся до истины! америка: факты и домыслы благодаря удивительному таланту снимать с себя любые кандалы гарри гудини стал одним из самых известных артистов двадцатого века. он выбирался из сейфов, банковских камер, фортепиано, чугунных котлов, гигантского футбольного мяча, даже из скелета таинственного морского чудища. осталась лишь одна сеть, откуда он не выбрался: паутина легенд, опутавшая его имя. фокусы скучнеют, когда вы знаете их объяснение. оказывается, всё просто. неинтересно. роб уильямс, артист но гудини – исключение, чем глубже копае

### Set of characters in the origial text

In [ ]:
print(f'Char\tDec\tHex\tPrintable?')
for i,c in enumerate(sorted(set(text))):
    print(f'{c}\t{ord(c)}\t{hex(ord(c))}\t{c.isprintable()}')

Char	Dec	Hex	Printable?

	10	0xa	False
 	32	0x20	True
!	33	0x21	True
,	44	0x2c	True
-	45	0x2d	True
.	46	0x2e	True
/	47	0x2f	True
0	48	0x30	True
1	49	0x31	True
2	50	0x32	True
4	52	0x34	True
5	53	0x35	True
7	55	0x37	True
8	56	0x38	True
9	57	0x39	True
:	58	0x3a	True
?	63	0x3f	True
 	160	0xa0	False
«	171	0xab	True
»	187	0xbb	True
А	1040	0x410	True
Б	1041	0x411	True
В	1042	0x412	True
Г	1043	0x413	True
Д	1044	0x414	True
Е	1045	0x415	True
З	1047	0x417	True
И	1048	0x418	True
Й	1049	0x419	True
К	1050	0x41a	True
Л	1051	0x41b	True
М	1052	0x41c	True
Н	1053	0x41d	True
О	1054	0x41e	True
П	1055	0x41f	True
Р	1056	0x420	True
С	1057	0x421	True
Т	1058	0x422	True
У	1059	0x423	True
Ф	1060	0x424	True
Х	1061	0x425	True
Ц	1062	0x426	True
Ч	1063	0x427	True
Ш	1064	0x428	True
Ы	1067	0x42b	True
Э	1069	0x42d	True
Я	1071	0x42f	True
а	1072	0x430	True
б	1073	0x431	True
в	1074	0x432	True
г	1075	0x433	True
д	1076	0x434	True
е	1077	0x435	True
ж	1078	0x436	True
з	1079	0x437	True
и	1080	0x438	True
й	1081	0x439	True
к	1082

### Set of characters in the cleaned text

In [ ]:
print(f'Char\tDec\tHex\tPrintable?')
for i,c in enumerate(sorted(set(russian_cleaner(text)))):
    print(f'{c}\t{ord(c)}\t{hex(ord(c))}\t{c.isprintable()}')

Char	Dec	Hex	Printable?
 	32	0x20	True
!	33	0x21	True
,	44	0x2c	True
-	45	0x2d	True
.	46	0x2e	True
:	58	0x3a	True
?	63	0x3f	True
а	1072	0x430	True
б	1073	0x431	True
в	1074	0x432	True
г	1075	0x433	True
д	1076	0x434	True
е	1077	0x435	True
ж	1078	0x436	True
з	1079	0x437	True
и	1080	0x438	True
й	1081	0x439	True
к	1082	0x43a	True
л	1083	0x43b	True
м	1084	0x43c	True
н	1085	0x43d	True
о	1086	0x43e	True
п	1087	0x43f	True
р	1088	0x440	True
с	1089	0x441	True
т	1090	0x442	True
у	1091	0x443	True
ф	1092	0x444	True
х	1093	0x445	True
ц	1094	0x446	True
ч	1095	0x447	True
ш	1096	0x448	True
щ	1097	0x449	True
ъ	1098	0x44a	True
ы	1099	0x44b	True
ь	1100	0x44c	True
э	1101	0x44d	True
ю	1102	0x44e	True
я	1103	0x44f	True
ё	1105	0x451	True
–	8211	0x2013	True


### Set of the removed/replaced characters

In [ ]:
print(f'Char\tDec\tHex\tPrintable?')
for i,c in enumerate(sorted( set(text).difference(set(russian_cleaner(text))))):
    print(f'{c}\t{ord(c)}\t{hex(ord(c))}\t{c.isprintable()}')

Char	Dec	Hex	Printable?

	10	0xa	False
/	47	0x2f	True
0	48	0x30	True
1	49	0x31	True
2	50	0x32	True
4	52	0x34	True
5	53	0x35	True
7	55	0x37	True
8	56	0x38	True
9	57	0x39	True
 	160	0xa0	False
«	171	0xab	True
»	187	0xbb	True
А	1040	0x410	True
Б	1041	0x411	True
В	1042	0x412	True
Г	1043	0x413	True
Д	1044	0x414	True
Е	1045	0x415	True
З	1047	0x417	True
И	1048	0x418	True
Й	1049	0x419	True
К	1050	0x41a	True
Л	1051	0x41b	True
М	1052	0x41c	True
Н	1053	0x41d	True
О	1054	0x41e	True
П	1055	0x41f	True
Р	1056	0x420	True
С	1057	0x421	True
Т	1058	0x422	True
У	1059	0x423	True
Ф	1060	0x424	True
Х	1061	0x425	True
Ц	1062	0x426	True
Ч	1063	0x427	True
Ш	1064	0x428	True
Ы	1067	0x42b	True
Э	1069	0x42d	True
Я	1071	0x42f	True
…	8230	0x2026	True
﻿	65279	0xfeff	False


In [ ]:
check_no_numbers(russian_cleaner(text))

[]

In [ ]:
text = '''Восклицательное предложение! А это какое? Инициалы -- не повод разрывать. Правда, А.С. Пушкин? -- Разумеется, голубчик. (Скобки оставляем.)'''

In [ ]:
sent_detector = nltk.data.load('tokenizers/punkt/russian.pickle',verbose=True,cache=False)

<<Loading nltk:tokenizers/punkt/PY3/russian.pickle>>


In [ ]:
sent_detector.tokenize((russian_cleaner(text)))

['восклицательное предложение!',
 'а это какое?',
 'инициалы -- не повод разрывать.',
 'правда, а.с. пушкин?',
 '-- разумеется, голубчик.',
 '(скобки оставляем.)']

In [ ]:
for sent in nltk.sent_tokenize(russian_cleaner(text), language="russian"):
    print(sent,"\n")

восклицательное предложение! 

а это какое? 

инициалы -- не повод разрывать. 

правда, а.с. пушкин? 

-- разумеется, голубчик. 

(скобки оставляем.) 



In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_text_norm.ipynb.
Converted 02_data.ipynb.
Converted 03_transcribe.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.
